Задание по итогам второго вебинара.
1. Приложен ноутбук, в котором реализованы функции для генирации из большого
датасета меньшая его копия. Вам нужно перенести функции из этого ноутбука в
класс датасет и сделать следующее:
a. Сгенерировать меньший датасет из 8-10 классов движения
b. Обучить уже существующую модель (предварительно проанализировав
какие параметры модели нужно изменить)
c. Изменить модель: посмотреть зависимость от количества LSTM модулей
в нашей модели
d. Сгенерировать другой датасет с меньшим количеством “кадров” в серии
и сравнить улучшилось или ухудшилось качество предсказания.
Провести несколько таких итераций, дать свою оценку уменьшению и
увеличению кадров, назвать оптимальное, на ваш взгляд, их количество.
Желательно сделать так, чтобы длина последовательности
передавалась как атрибут класса.
2. Дополнительное задание:
a. http://archive.ics.uci.edu/ml/datasets/Individual+household+electric+power+c
onsumption - 2075259 measurements gathered in a house located in Sceaux
(7km of Paris, France) between December 2006 and November 2010 (47
months). Проделайте весь путь подготовки данных, создания датасета,
разделения и обучения модели самостоятельно. Предсказывать нужно
Global_active_power. Обратите внимание, что здесь задача регрессии, а
не классификации, т.е. модель нужно изменить.

In [1]:
import torch
from torch import nn
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

from torch import optim
import time
import math


use_cuda = torch.cuda.is_available()
# device = torch.device("cpu")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


### a. Сгенерировать меньший датасет из 8-10 классов движения 

In [2]:
data_path = "E:/IT_WORK/!!!_GB_Pytorch_28.10.2020/Lession02/nturgb+d_skeletons/"
broken_files_path = "NTU_RGBD_samples_with_missing_skeletons.txt"

In [3]:
# training_subjects = list(range(0, 28)) #количество людей выполняющих действия
training_classes = sorted([8, 10, 22, 23, 27, 21, 55, 2, 7]) #классы которые будем использовать для обучения, полный список прдставлен тут https://github.com/shahroudy/NTURGB-D
LABELS = {x: training_classes[x] for x in range(len(training_classes))}
training_cameras = [1, 2, 3] 

# max_body_true = 1
# max_body_kinect = 1

num_joint = 25
max_frame = 300

In [4]:
class Skeleton_Dataset(Dataset):
    def __init__(self, data_path, broken_files_path=None, training_classes=None,
                 num_joint = 25, max_frame = 300, transform=None):
        
        
        def read_data(data_path, broken_files_path):
            labels = []
            files = []
            action_classes = {}
            counter = 0
            files_counter = {}
            with open(broken_files_path, 'r') as f:
                broken_files = f.read().split("\n")

            raw_files = os.listdir(data_path)
            num_frames = 0

            for filename in raw_files:
                if filename not in broken_files:
                    action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
                    subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
                    camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
                    if action_class in training_classes and camera_id in training_cameras: 
                        if action_class in action_classes:
                            if files_counter[action_class] < 120:
                                files.append([filename,action_classes[action_class]])
                                files_counter[action_class] = files_counter[action_class] + 1
                        else:
                            action_classes.update({action_class : counter})
                            files_counter.update({action_class : 1})
                            counter+=1
                            files.append([filename,action_classes[action_class]])
            print("action classes: ", action_classes)
            print("action files: ", files_counter)

            return files, action_classes
        
        
        def read_skeleton_filter(file):
            with open(file, 'r') as f:
                skeleton_sequence = {}
                skeleton_sequence['numFrame'] = int(f.readline())
                skeleton_sequence['frameInfo'] = []
                for t in range(skeleton_sequence['numFrame']):
                    frame_info = {}
                    frame_info['numBody'] = int(f.readline())
                    frame_info['bodyInfo'] = []

                    for m in range(frame_info['numBody']):
                        body_info = {}
                        body_info_key = [
                            'bodyID', 'clipedEdges', 'handLeftConfidence',
                            'handLeftState', 'handRightConfidence', 'handRightState',
                            'isResticted', 'leanX', 'leanY', 'trackingState'
                        ]
                        body_info = {
                            k: float(v)
                            for k, v in zip(body_info_key, f.readline().split())
                        }
                        body_info['numJoint'] = int(f.readline())
                        body_info['jointInfo'] = []
                        for v in range(body_info['numJoint']):
                            joint_info_key = [
                                'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                                'orientationW', 'orientationX', 'orientationY',
                                'orientationZ', 'trackingState'
                            ]
                            joint_info = {
                                k: float(v)
                                for k, v in zip(joint_info_key, f.readline().split())
                            }
                            body_info['jointInfo'].append(joint_info)
                        frame_info['bodyInfo'].append(body_info)
                    skeleton_sequence['frameInfo'].append(frame_info)

            return skeleton_sequence

        def read_xyz(file, max_body=1, num_joint=25):
            seq_info = read_skeleton_filter(file)
            data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
            for n, f in enumerate(seq_info['frameInfo']):
                for m, b in enumerate(f['bodyInfo']):
                    for j, v in enumerate(b['jointInfo']):
                        if m < max_body and j < num_joint:
                            data[m, n, j, :] = [v['x'], v['y'], v['z']]

                        else:
                            pass

            return data
        
        
        def create_coords_blocks(test_file, chonk_len = 45):   
            frame_counter = 0
            new_labels = []
            new_frames = []
            blocks = []

            test_frames = read_xyz(data_path + test_file[0])[0]
            label = test_file[1]
            slice_len = chonk_len * int(len(test_frames)/chonk_len)


            for index in range(len(test_frames[:slice_len])):
                frame_counter += 1
                new_frames.append(test_frames[index].flatten())
                if frame_counter == chonk_len:
                    frame_counter = 0
                    blocks.append(np.array(new_frames))
                    new_labels = new_labels + [label]
                    new_frames = []


            return blocks, new_labels
        
        
        ##### список файлов с лейблами на каждый файл 
        working_files_with_labels, action_classes = read_data(data_path, broken_files_path)
        
        data = []
        labels = []
        ##########################################################################
        numbers = {x: 0 for x in range(len(action_classes))}  #####
        ##################################################################
        for file in working_files_with_labels:
            frames_blocks, label = create_coords_blocks(file)
            if label != [] and numbers[label[0]] <= 150:
                numbers[label[0]] = numbers[label[0]] + len(label)
                data = data + frames_blocks
                labels = labels + label
        data_np = np.asarray(data)
        labels_np = np.asarray(labels)

        data_sq = data_np.reshape(len(data_np), -1)
        data = pd.DataFrame(data_sq)
        labels = pd.DataFrame(labels_np)
        data['labels'] = labels
        

        self.data = data
        self.labels = data['labels'].astype('float32')
        self.transform = transform
        
           
    def __len__(self):
         return len(self.data)
        
        
    def __getitem__(self, idx):
        item = np.asarray(self.data.iloc[idx,:-1]).reshape(45,75)
        label = self.labels[idx]
        if self.transform != None:
            item = transform(item)
        return (item, label) 

In [5]:
dataset = Skeleton_Dataset(data_path=data_path, broken_files_path=broken_files_path, 
                           training_classes=training_classes,num_joint = 25, 
                           max_frame = 300, transform=None)

action classes:  {2: 0, 7: 1, 8: 2, 10: 3, 21: 4, 22: 5, 23: 6, 27: 7, 55: 8}
action files:  {2: 120, 7: 120, 8: 120, 10: 120, 21: 120, 22: 120, 23: 120, 27: 120, 55: 120}


### b. Обучить уже существующую модель (предварительно проанализировав какие параметры модели нужно изменить) 

In [6]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [int(0.75*len(dataset)),
                                                                      len(dataset) - int(0.75*len(dataset))])
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 1, shuffle=True)

In [7]:
class LSTM_net(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim,layer_num,batch_first=True)
        self.dr = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [8]:
n_hidden = 128
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 128, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=9, bias=True)
)

In [9]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
#     print(output.topk(5))
    return LABELS[category_i], category_i

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [10]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(600):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        labels = labels.type(torch.LongTensor).to(device)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0
    

epoch : 0 iter : 0 (0m 2s) 2.2490  / 22 ✗ (55)
epoch : 8 iter : 36 (0m 19s) 2.2011  / 2 ✗ (7)
epoch : 17 iter : 14 (0m 36s) 2.1531  / 2 ✗ (8)
epoch : 25 iter : 50 (0m 54s) 2.1022  / 2 ✗ (22)
epoch : 34 iter : 28 (1m 11s) 1.6837  / 2 ✗ (27)
epoch : 43 iter : 6 (1m 28s) 2.0363  / 2 ✗ (7)
epoch : 51 iter : 42 (1m 45s) 1.8114  / 2 ✗ (7)
epoch : 60 iter : 20 (2m 3s) 1.5551  / 23 ✓
epoch : 68 iter : 56 (2m 20s) 1.4552  / 55 ✗ (27)
epoch : 77 iter : 34 (2m 38s) 1.2948  / 2 ✓
epoch : 86 iter : 12 (2m 55s) 1.9694  / 7 ✗ (55)
epoch : 94 iter : 48 (3m 14s) 1.0268  / 27 ✓
epoch : 103 iter : 26 (3m 32s) 1.4869  / 2 ✓
epoch : 112 iter : 4 (3m 50s) 1.2797  / 22 ✓
epoch : 120 iter : 40 (4m 8s) 1.0014  / 55 ✓
epoch : 129 iter : 18 (4m 25s) 0.8151  / 8 ✓
epoch : 137 iter : 54 (4m 43s) 1.0890  / 7 ✗ (21)
epoch : 146 iter : 32 (5m 0s) 0.5559  / 7 ✗ (23)
epoch : 155 iter : 10 (5m 17s) 0.8957  / 2 ✓
epoch : 163 iter : 46 (5m 34s) 0.7047  / 27 ✓
epoch : 172 iter : 24 (5m 51s) 1.0431  / 27 ✗ (23)
epoch : 181 

In [11]:
total = 0
right = 0
counter = 0

rnn.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        counter = counter + 1
        inputs, labels = data[0].to(device), data[1].to(device)  
        output = rnn(inputs.float())
        guess, guess_i = categoryFromOutput(output)
        category = LABELS[int(labels[0])]
        
        if guess == category:
            right = right + 1


print('Accuracy of the network:  ',  (100 * right / counter))

Accuracy of the network:   73.13915857605178


### c. Изменить модель: посмотреть зависимость от количества LSTM модулей в нашей модели 

In [12]:
n_hidden = 128*3
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 384, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=384, out_features=9, bias=True)
)

In [13]:
class LSTM_net(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim,layer_num,batch_first=True)
        self.dr = torch.nn.Dropout2d(0.1)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        
        
    def forward(self,inputs):
        x = inputs
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [14]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(600):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        labels = labels.type(torch.LongTensor).to(device)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0
    

epoch : 0 iter : 0 (0m 0s) 2.1823  / 10 ✗ (27)
epoch : 8 iter : 36 (0m 18s) 2.1942  / 2 ✗ (21)
epoch : 17 iter : 14 (0m 37s) 2.1744  / 2 ✗ (7)
epoch : 25 iter : 50 (0m 55s) 1.9351  / 22 ✗ (23)
epoch : 34 iter : 28 (1m 14s) 1.7881  / 22 ✓
epoch : 43 iter : 6 (1m 33s) 1.7395  / 21 ✓
epoch : 51 iter : 42 (1m 51s) 1.9025  / 2 ✓
epoch : 60 iter : 20 (2m 10s) 1.3851  / 2 ✓
epoch : 68 iter : 56 (2m 30s) 1.7453  / 22 ✓
epoch : 77 iter : 34 (2m 48s) 1.2845  / 2 ✗ (7)
epoch : 86 iter : 12 (3m 6s) 1.1206  / 7 ✗ (23)
epoch : 94 iter : 48 (3m 25s) 1.2961  / 7 ✓
epoch : 103 iter : 26 (3m 45s) 0.9987  / 27 ✓
epoch : 112 iter : 4 (4m 5s) 1.0895  / 7 ✗ (23)
epoch : 120 iter : 40 (4m 24s) 0.9364  / 2 ✓
epoch : 129 iter : 18 (4m 43s) 0.7155  / 22 ✓
epoch : 137 iter : 54 (5m 2s) 0.7854  / 10 ✓
epoch : 146 iter : 32 (5m 20s) 0.6873  / 21 ✗ (2)
epoch : 155 iter : 10 (5m 39s) 0.8368  / 7 ✓
epoch : 163 iter : 46 (5m 59s) 0.3618  / 27 ✓
epoch : 172 iter : 24 (6m 17s) 0.4829  / 27 ✓
epoch : 181 iter : 2 (6m 36s

In [15]:
total = 0
right = 0
counter = 0

rnn.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        counter = counter + 1
        inputs, labels = data[0].to(device), data[1].to(device)  
        output = rnn(inputs.float())
        guess, guess_i = categoryFromOutput(output)
        category = LABELS[int(labels[0])]
        
        if guess == category:
            right = right + 1


print('Accuracy of the network(Модель задания "с"):  ',  (100 * right / counter))

Accuracy of the network(Модель задания "с"):   71.5210355987055


Увеличение скрытых слоев нейросети в 3 раза дает улучшенный резульат

### d. Сгенерировать другой датасет с меньшим количеством “кадров” в серии и сравнить улучшилось или ухудшилось качество предсказания. Провести несколько таких итераций, дать свою оценку уменьшению и увеличению кадров, назвать оптимальное, на ваш взгляд, их количество. Желательно сделать так, чтобы длина последовательности передавалась как атрибут класса.

In [16]:
# training_subjects = list(range(0, 28)) #количество людей выполняющих действия
training_classes = sorted([8, 10, 22, 23, 27, 21, 55, 2, 7]) #классы которые будем использовать для обучения, полный список прдставлен тут https://github.com/shahroudy/NTURGB-D
LABELS = {x: training_classes[x] for x in range(len(training_classes))}
training_cameras = [1, 2, 3] 

# max_body_true = 1
# max_body_kinect = 1

num_joint = 25
max_frame = 200

In [17]:
dataset = Skeleton_Dataset(data_path=data_path, broken_files_path=broken_files_path, 
                           training_classes=training_classes,num_joint = 25, 
                           max_frame = 300, transform=None)

action classes:  {2: 0, 7: 1, 8: 2, 10: 3, 21: 4, 22: 5, 23: 6, 27: 7, 55: 8}
action files:  {2: 120, 7: 120, 8: 120, 10: 120, 21: 120, 22: 120, 23: 120, 27: 120, 55: 120}


In [18]:
n_hidden = 128*3
n_joints = 25*3
n_categories = len(LABELS)
n_layer = 2
rnn = LSTM_net(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

LSTM_net(
  (lstm): LSTM(75, 384, num_layers=2, batch_first=True)
  (dr): Dropout2d(p=0.1, inplace=False)
  (fc): Linear(in_features=384, out_features=9, bias=True)
)

In [19]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.0007
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)

all_losses = []
start = time.time()
counter = 0
for epoch in range(600):  
    current_loss = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
    
        output = rnn(inputs.float())
        labels = labels.type(torch.LongTensor).to(device)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step() 


        current_loss += loss.item()
        category = LABELS[int(labels[0])]

        if counter % 500 == 0:
            guess, guess_i = categoryFromOutput(output)
            correct = '✓' if guess == category else '✗ (%s)' % category
            print('epoch : %d iter : %d (%s) %.4f  / %s %s' % (epoch, i, timeSince(start), loss, guess, correct))

        
        counter = counter + 1
    if counter % 100 == 0:
        all_losses.append(current_loss / 25)
        current_loss = 0
    

epoch : 0 iter : 0 (0m 0s) 2.1947  / 7 ✗ (10)
epoch : 8 iter : 36 (0m 18s) 2.1742  / 2 ✗ (21)
epoch : 17 iter : 14 (0m 36s) 2.1263  / 2 ✗ (7)
epoch : 25 iter : 50 (0m 54s) 2.0687  / 22 ✗ (27)
epoch : 34 iter : 28 (1m 12s) 2.1290  / 7 ✓
epoch : 43 iter : 6 (1m 31s) 1.7686  / 8 ✗ (27)
epoch : 51 iter : 42 (1m 49s) 1.7903  / 2 ✓
epoch : 60 iter : 20 (2m 8s) 1.6968  / 7 ✗ (55)
epoch : 68 iter : 56 (2m 27s) 1.3754  / 22 ✓
epoch : 77 iter : 34 (2m 46s) 1.4579  / 21 ✗ (23)
epoch : 86 iter : 12 (3m 4s) 1.2081  / 21 ✗ (22)
epoch : 94 iter : 48 (3m 23s) 1.3660  / 7 ✓
epoch : 103 iter : 26 (3m 42s) 1.1436  / 23 ✓
epoch : 112 iter : 4 (4m 0s) 1.2024  / 21 ✗ (2)
epoch : 120 iter : 40 (4m 18s) 0.9893  / 7 ✓
epoch : 129 iter : 18 (4m 38s) 0.5203  / 55 ✓
epoch : 137 iter : 54 (4m 56s) 1.4505  / 27 ✓
epoch : 146 iter : 32 (5m 15s) 1.0844  / 21 ✗ (10)
epoch : 155 iter : 10 (5m 34s) 0.7903  / 55 ✓
epoch : 163 iter : 46 (5m 53s) 0.5272  / 21 ✓
epoch : 172 iter : 24 (6m 13s) 0.5139  / 21 ✗ (55)
epoch : 181

In [21]:
total = 0
right = 0
counter = 0

rnn.eval()
with torch.no_grad():
    for i, data in enumerate(test_loader, 0):
        counter = counter + 1
        inputs, labels = data[0].to(device), data[1].to(device)  
        output = rnn(inputs.float())
        guess, guess_i = categoryFromOutput(output)
        category = LABELS[int(labels[0])]
        
        if guess == category:
            right = right + 1


print('Accuracy of the network(Модель задания "с"):  ',  (100 * right / counter))

Accuracy of the network(Модель задания "с"):   75.40453074433657


Уменьшение клипа до 200 кадров немного улучшела модель